In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

Best time to buy and sell stocks - I

In [5]:
def stocks1(prices: list[int]) -> int:
    min_, max_profit = prices[0], 0
    for price in prices:
        max_profit = max(max_profit, price - min_)
        min_ = min(min_, price)

    return max_profit

# Testing the solution
assert stocks1([7,1,5,3,6,4]) == 5
assert stocks1([7,6,4,3,1]) == 0

Weekly Contest 398: 19th May 2024

In [6]:
# Q1
def isArraySpecial(nums: list[int]) -> bool:
    N = len(nums)
    for i in range(N - 1):
        i1, i2 = nums[i] % 2, nums[i + 1] % 2
        if (i1 == 0 and i2 == 0) or (i1 == 1 and i2 == 1):
            return False
    else:
        return True

# Testing the solution
assert isArraySpecial([4,3,1,6]) == False
assert isArraySpecial([2,1,4]) == True
assert isArraySpecial([1]) == True

In [7]:
# Q2
def isArraySpecial2(nums: list[int], queries: list[list[int]]) -> list[bool]:
    N = len(nums)
    parity: list[int] = list(map(lambda x: x % 2, nums))

    @functools.cache
    def backtrack(start: int, end: int) -> bool:
        if start == end:
            return True
        else:
            mid = (start + end) // 2
            isSpecial =  parity[mid] != parity[mid + 1] and backtrack(start, mid) and backtrack(mid + 1, end)
            return isSpecial

    result: list[bool] = []
    for i, j in queries:
        result.append(backtrack(i, j))

    return result

# Testing the solution
assert isArraySpecial2([4,3,1,6], [[0,2],[2,3]]) == [False, True]
assert isArraySpecial2([3,4,1,2,6], [[0,4]]) == [False]

In [8]:
# Q3
def sumDigitDifferencesBrute(nums: list[int]) -> int:
    length = len(str(nums[0]))
    count = 0
    for n1, n2 in itertools.combinations(nums, r=2):
        n1_str, n2_str = str(n1), str(n2)
        for i in range(length):
            if n1_str[i] != n2_str[i]:
                count += 1

    return count

def sumDigitDifferences(nums: list[int]) -> int:
    def count(arr: list[int]) -> int:
        N = len(arr)
        temp = 0
        for i in range(N - 1):
            temp += arr[i] * sum(arr[i + 1:])

        return temp

    length = len(str(nums[0]))
    digits_dict: dict[int, dict[str, int]] = dict()
    for n in nums:
        n_str = str(n)
        for i in range(length):
            positions = digits_dict.get(i, dict())
            positions[n_str[i]] = positions.get(n_str[i], 0) + 1
            digits_dict[i] = positions

    result: int = 0
    for i in range(length):
        result += count(list(digits_dict[i].values()))

    return result

# Testing the solution
for i in range(100):
    temp = list(np.random.randint(10, 100, 50))
    assert sumDigitDifferencesBrute(temp) == sumDigitDifferences(temp)

Sum of XOR totals
Easy LC: https://leetcode.com/problems/sum-of-all-subset-xor-totals

In [9]:
# https://leetcode.com/problems/sum-of-all-subset-xor-totals/submissions/1263288868
def subsetXORSumMemo(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, curr: int) -> int:
        if i == N:
            return curr
        else:
            return backtrack(i + 1, curr ^ nums[i]) + backtrack(i + 1, curr)

    N = len(nums)
    return backtrack(0, 0)

# Testing the solution
assert subsetXORSumMemo([1,3]) == 6
assert subsetXORSumMemo([5,1,6]) == 28

Largest Rectangle in a histogram

In [10]:
# https://leetcode.com/problems/largest-rectangle-in-histogram/submissions/1266278725
def largestRectangleArea(heights: list[int]) -> int:
    """
    Count the number of rectangles to the right that are greater than or equal to self
    Count the number of rectangles to the left that are greater than or equal to self
    """

    N = len(heights)

    right_dp: list[int] = [N - i for i in range(N)]
    left_dp: list[int] = [i + 1 for i in range(N)]
    stack: list[tuple[int, int]] = []

    for i in range(N):
        curr: tuple[int, int] = (heights[i], i)
        while stack and stack[-1][0] > curr[0]:
            prev = stack.pop()
            right_dp[prev[1]] = curr[1] - prev[1]
        stack.append(curr)

    stack.clear()

    for i in range(N - 1, -1, -1):
        curr = (heights[i], i)
        while stack and stack[-1][0] > curr[0]:
            prev = stack.pop()
            left_dp[prev[1]] = prev[1] - curr[1]
        stack.append(curr)

    max_ = 0
    for i in range(N):
        max_ = max(max_, (left_dp[i] + right_dp[i] - 1) * heights[i])

    return max_

# Testing the solution
assert largestRectangleArea([2,1,5,6,2,3]) == 10
assert largestRectangleArea([2,4]) == 4
assert largestRectangleArea([2,1,2]) == 3

Container Queries - CN Weekly - 3

In [11]:
def containerQueries(N: int, S: str, query_len: int, queries: list[list[int]]) -> list[int]:
    positions: dict[str, collections.deque[int]] = dict()
    for i in range(N):
        indices = positions.get(S[i], collections.deque())
        indices.append(i)
        positions[S[i]] = indices

    end_pos = 0
    for k in positions:
        end_pos = max(end_pos, positions[k].popleft())

    dp: list[int] = [end_pos]
    for i in range(1, N):
        prev = S[i - 1]
        next_pos = max(dp[-1], positions[prev].popleft() if positions[prev] else N)
        dp.append(next_pos)

    result: list[int] = []
    for i, j in queries:
        if j >= dp[i]:
            result.append(1)
        else:
            result.append(0)

    return result

# Testing the solution
assert containerQueries(8, "yukppuyk", 2, [[0,4],[5,7]]) == [1, 0]